# Notebook: Kiểm tra giới hạn page_size của API 

## Vì sao cần kiểm tra `page_size`?

Trong giai đoạn khảo sát API, ta phải kiểm tra giá trị `page_size` tối đa mà Pancake POS hỗ trợ.  
Mục tiêu của việc này:

1. **Xác định giới hạn của API**  
   - Tài liệu API có thể không ghi rõ `page_size` tối đa.  
   - Nếu gửi quá lớn, API có thể trả về lỗi (400/500) hoặc cắt bớt dữ liệu.

2. **Tối ưu hiệu suất ETL**  
   - `page_size` càng lớn → số lần gọi API càng ít → tải dữ liệu nhanh hơn.  
   - Nhưng nếu quá lớn → dễ bị timeout hoặc nghẽn mạng.  

3. **Giảm nguy cơ chạm rate limit**  
   - API thường giới hạn số lần gọi trong 1 phút/giờ.  
   - `page_size` nhỏ → nhiều request → dễ vượt giới hạn.  
   - `page_size` hợp lý → ít request hơn → tối ưu quota.

4. **Đảm bảo dữ liệu đầy đủ và chính xác**  
   - Nếu không kiểm tra, có thể mất dữ liệu (API tự cắt kết quả).  
   - Ví dụ: yêu cầu `page_size=2000` nhưng API chỉ trả tối đa `1000`.  

**Kết luận**: Việc test `page_size` giúp tìm ra “ngưỡng vàng” để vừa giảm số lần gọi API, vừa đảm bảo dữ liệu đầy đủ, không bị lỗi.



---
### 1.1 Import thư viện & load biến môi trường
### Giải thích
- Import thư viện: `requests` để gọi API, `dotenv` để load biến môi trường.  
- Đọc `API_KEY` và `SHOP_ID` từ file `.env`.  

In [1]:
import requests
from dotenv import load_dotenv
import os

# Load biến môi trường từ file .env
load_dotenv()
API_KEY = os.getenv("API_KEY")
SHOP_ID = os.getenv("SHOP_ID")


### 1.2. Định nghĩa hàm test_page_size
### Giải thích
- `test_page_size` nhận `shop_id` và `page_size` làm tham số.  
- Gọi API `/orders` với `page=1` và `page_size` tùy chọn.  
- In ra:
  - `status code` của API.  
  - Số lượng bản ghi lấy được (nếu thành công).  
  - Một phần thông báo lỗi (nếu thất bại).  


In [2]:
def test_page_size(shop_id, page_size):
    url = f"https://pos.pages.fm/api/v1/shops/{shop_id}/orders"
    params = {"api_key": API_KEY, "page": 1, "page_size": page_size}
    resp = requests.get(url, params=params)
    print(f"Page size {page_size} → status {resp.status_code}")
    
    if resp.status_code == 200:
        data = resp.json().get("data", [])
        print(f"👉 Lấy được {len(data)} bản ghi")
    else:
        print("❌ Lỗi:", resp.text[:200])

### 1.3. Thử nghiệm với nhiều giá trị page_size
### Giải thích
- Thử gọi API với các giá trị `page_size` khác nhau: 50, 100, 200, 500, 1000, 2000.  
- Quan sát để xác định ngưỡng tối đa mà API hỗ trợ.  


In [4]:
for ps in [50, 100, 200, 500, 1000, 2000]:
    test_page_size(SHOP_ID, ps)

Page size 50 → status 200
👉 Lấy được 50 bản ghi
Page size 100 → status 200
👉 Lấy được 100 bản ghi
Page size 200 → status 200
👉 Lấy được 200 bản ghi
Page size 500 → status 200
👉 Lấy được 500 bản ghi
Page size 1000 → status 200
👉 Lấy được 1000 bản ghi
Page size 2000 → status 422
❌ Lỗi: {"error":"page size must be less than or equal to 1000"}


 **Kết luận:** Pancake POS API cho phép tối đa **1000 bản ghi mỗi lần gọi** (`page_size ≤ 1000`).  
- Khi ETL, nên đặt `page_size = 1000` để **giảm số lần request**, tối ưu tốc độ tải dữ liệu.  
- Với dataset rất lớn, vẫn cần **phân trang (page=1,2,3,...)** nhưng mỗi trang có thể lấy tối đa 1000 bản ghi.  
